# Text Generation: How to run inference on the endpoint you have created?

In [25]:
import json
import boto3

### Query endpoint that you have created

In [26]:
endpoint_name = "jumpstart-dft-hf-llm-mistral-7b-instruct"


def query_endpoint(payload):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(payload).encode("utf-8")
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

### Supported parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments.
***

In [27]:
from typing import Dict, List


def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for user, answer in zip(instructions[::2], instructions[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)


def print_instructions(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}\n\n{bold}> Output{unbold}\n{response[0]['generated_text']}\n")

In [28]:
instructions = [{"role": "user", "content": "what is the recipe of mayonnaise?"}]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = query_endpoint(payload)
print_instructions(prompt, response)

> Input
<s>[INST] what is the recipe of mayonnaise? [/INST] 

> Output
1. Begin by whisking together 1 tablespoon of water, 1 tablespoon of apple cider vinegar, 1 tablespoon of dijon mustard, and 2 egg yolks until they are fully combined.
2. Slowly pour in 1 cup of oil, starting with 2 tablespoons at a time. Whisk the mixture well after each addition.
3. Once all the oil has been added, whisk the mayonnaise mixture until it is smooth and creamy, and you can also add salt and pepper to taste.
4. If the ormayonnaise seems too thick, you can add more water one tablespoon at a time until you achieve the desired consistency.
5. Store the homemade mayonnaise in an airtight container in the refrigerator until ready to use.

Alternatively, you can also make a vegan version of mayonnaise by substituting the egg yolks with silken tofu or mashed avocado, and you can also use lemon juice or apple cider vinegar instead of water. But keep in mind that oil is still an important component for a good m

In [29]:
instructions = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
    },
    {"role": "user", "content": "What is so great about #1?"},
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = query_endpoint(payload)
print_instructions(prompt, response)

> Input
<s>[INST] I am going to Paris, what should I see? [/INST] Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.</s><s>[INST] What is so great about #1? [/INST] 

> Output
1. The Eiffel Tower 

In [30]:
instructions = [
    {
        "role": "user",
        "content": "In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month?",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
}
response = query_endpoint(payload)
print_instructions(prompt, response)

> Input
<s>[INST] In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month? [/INST] 

> Output
1. Use the `find` command with the `-type f` option to search for files only, and the `-mtime -30` option to search for files modified in the last 30 days (1 month).
2. Use the `-name` option to search for files with a `.txt` extension.
3. Use the `-print` option to print the file names.

The command would be:

```bash
find . -type f -mtime -30 -name "*.txt" -print
```

This will list all text files in the current directory (excluding subdirectories) that have been modified in the last month.



In [31]:
instructions = [
    {
        "role": "user",
        "content": "What is the difference between inorder and preorder traversal? Give an example in Python.",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
}
response = query_endpoint(payload)
print_instructions(prompt, response)

> Input
<s>[INST] What is the difference between inorder and preorder traversal? Give an example in Python. [/INST] 

> Output
1. Inorder traversal: Inorder traversal is a way of visiting nodes in a binary tree such that the left subtree is visited before the root node and the right subtree is visited after the root node. This is done in the following order: left subtree, root node, right subtree.

Example in Python:

```python
class Node:
    def __init__(self, key):
        self.left = None
        self.right = None
        self.val = key

def inorder_traversal(root):
    if root:
        inorder_traversal(root.left)
        print(root.val, end=" ")
        inorder_traversal(root.right)

# Driver code
root = Node(50)
root.left = Node(30)
root.right = Node(20)
root.left.left = Node(10)
root.left.right = Node(40)

inorder_traversal(root)
```
Output:
```
10 20 30 40 50



## Importing Dataset

In [32]:
import pandas as pd

bucket='s3://complaints.csv'
data_key = 'complaints.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

data = pd.read_csv(data_location)

In [33]:
data.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/26/18,Credit card or prepaid card,General-purpose credit card or charge card,Fees or interest,Unexpected increase in interest rate,NaN,None,JPMORGAN CHASE & CO.,FL,32225,None,Consent not provided,Web,4/26/18,Closed with explanation,Yes,NaN,2887783
1,7/18/23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,NaN,None,CAPITAL ONE FINANCIAL CORPORATION,GA,30082,Older American,Consent not provided,Web,7/18/23,Closed with non-monetary relief,Yes,NaN,7276310
2,11/17/17,Checking or savings account,Checking account,Managing an account,Cashing a check,NaN,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",MA,1118,None,NaN,Referral,11/22/17,Closed with explanation,Yes,NaN,2736589
3,1/21/21,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Fraud or scam,None,On XX/XX/20 {$33000.00} was wired from my bank...,None,JPMORGAN CHASE & CO.,FL,32137,None,Consent provided,Web,1/21/21,Closed with explanation,Yes,NaN,4082404
4,9/23/14,Bank account or service,Checking account,"Account opening, closing, or management",None,NaN,None,WELLS FARGO & COMPANY,TN,37660,Servicemember,NaN,Phone,9/29/14,Closed with explanation,Yes,No,1040743
5,8/23/20,Credit card or prepaid card,General-purpose credit card or charge card,Getting a credit card,Card opened as result of identity theft or fraud,NaN,None,CAPITAL ONE FINANCIAL CORPORATION,FL,32607,None,Consent not provided,Web,8/23/20,Closed with explanation,Yes,NaN,3809841
6,7/26/22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",PA,19405,Servicemember,Consent not provided,Web,7/26/22,Closed with explanation,Yes,NaN,5809964
7,2/14/23,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,None,CAPITAL ONE FINANCIAL CORPORATION,PA,19380,Servicemember,Consent not provided,Web,2/14/23,Closed with explanation,Yes,NaN,6573129
8,5/22/19,Mortgage,Conventional home mortgage,Trouble during payment process,None,"Wells Fargo purchased my mortgage from XXXX, s...",Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,MO,63376,None,Consent provided,Web,5/22/19,Closed with explanation,Yes,NaN,3250784
9,5/4/23,Checking or savings account,Checking account,Problem with a lender or other company chargin...,Transaction was not authorized,I am writing regarding unauthorized ACH transa...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",PA,152XX,None,Consent provided,Web,5/4/23,Closed with explanation,Yes,NaN,6930305


In [34]:
complaints = data.dropna(subset=['Consumer complaint narrative'])

In [35]:
complaints.isnull().sum()

Date received                        0
Product                              0
Sub-product                          0
Issue                                0
Sub-issue                            0
Consumer complaint narrative         0
Company public response              0
Company                              0
State                                0
ZIP code                             0
Tags                                 0
Consumer consent provided?           0
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?              148723
Complaint ID                         0
dtype: int64

In [36]:
complaints_revised = complaints[['Company', 'Consumer complaint narrative']]

In [37]:
company_list = ['WELLS FARGO & COMPANY', 'BANK OF AMERICA, NATIONAL ASSOCIATION', 'CAPITAL ONE FINANCIAL CORPORATION', 'JPMORGAN CHASE $ CO.', 'CITIBANK, N.A.']
complaints_revised = complaints_revised[complaints_revised['Company'].isin(company_list)]
complaints_revised.head(10)

,Company,Consumer complaint narrative
8,WELLS FARGO & COMPANY,"Wells Fargo purchased my mortgage from XXXX, s..."
9,"BANK OF AMERICA, NATIONAL ASSOCIATION",I am writing regarding unauthorized ACH transa...
10,WELLS FARGO & COMPANY,This happens every time I re-order checks from...
11,"BANK OF AMERICA, NATIONAL ASSOCIATION",In XX/XX/2022 I was charged for a hotel room t...
15,CAPITAL ONE FINANCIAL CORPORATION,Capital one/XXXX XXXX open account XXXX XX/XX...
20,"CITIBANK, N.A.","XXXX XXXX XXXX XXXX XXXX XXXX, PA XXXX XXXX 15..."
32,"BANK OF AMERICA, NATIONAL ASSOCIATION",On XXXX 2023 I XXXX XXXX went into the XXXX XX...
35,CAPITAL ONE FINANCIAL CORPORATION,The presence of a derogatory rating on my acco...
37,"CITIBANK, N.A.",I believe someone cashed a fraudulent check on...
39,WELLS FARGO & COMPANY,Called Wells Fargo regarding fraudulent charge...


In [38]:
import random

# Specify the sample size you want - 50
sample_size = 50

# Extract the 'Consumer complaint narrative' column as a list
narratives = complaints_revised['Consumer complaint narrative'].tolist()

# Check if the number of narratives is greater than the desired sample size
if len(narratives) > sample_size:
    random_samples = random.sample(narratives, sample_size)
else:
    random_samples = narratives

# Create a DataFrame from the sampled narratives
sample = pd.DataFrame({'Consumer complaint narrative': random_samples})

In [39]:
sample = sample.dropna(subset=['Consumer complaint narrative'])

sample.isnull().sum()

## Synthesized Customer Complaints

In [44]:
random_samples[:5]

['This is a follow-up of complaint XXXX. FIA Card Services has never sent me their definition of " cash equivalent \'\' or anything else. In addition, to harass me further, the last statement included a " minimum interest charge \'\' of {$1.00}. I owe them nothing and that charge was so absurd that their automated line took it off. FIA cheat and lies and is totally irresponsible. There should be some penalties or they should at least tell the truth. They record their calls so al this can be easily verified.',
 'Hi. \nMonth ago I opened Bank of America cash reward credit card. I got card and start to use it. \nI made purchases at gas station, grocery stores and online. Already get cash reward ( {$110.00} ) in my account. By end of billing cycle I got XXXX and paid in full total balance in the local branch by next day. Next day I got letter from Bank of America stated that my account has been closed without any explanations.When I called to a toll free number listed in the letter, automa

In [41]:
#Robustness 
for i in range(len(random_samples[0:5])):
    instructions = [
        {
            "role": "Your task is to determine if customer complaints from banks are well written without any issues Provide modified samples.", 
            
            "content": f"""
            Determine if complaints are well-written (considered desirable), exhibit severe grammar or punctuation issues (considered problematic), or contain irrelevant, frivolous information (also considered problematic).
            Make your response as short as possible.
            
            Complaints: '''{random_samples[i]}'''"""
        } 
    ]
    prompt = format_instructions(instructions)
    payload = {
        "inputs": prompt, 
        "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
    }
    response = query_endpoint(payload)
    print_instructions(prompt, response)

> Input
<s>[INST] Determine if complaints are well-written (considered desirable), exhibit severe grammar or punctuation issues (considered problematic), or contain irrelevant, frivolous information (also considered problematic).
            Make your response as short as possible.
            
            Complaints: '''This is a follow-up of complaint XXXX. FIA Card Services has never sent me their definition of " cash equivalent '' or anything else. In addition, to harass me further, the last statement included a " minimum interest charge '' of {$1.00}. I owe them nothing and that charge was so absurd that their automated line took it off. FIA cheat and lies and is totally irresponsible. There should be some penalties or they should at least tell the truth. They record their calls so al this can be easily verified.''' [/INST] 

> Output


The complaint is well-written and contains relevant information.

> Input
<s>[INST] Determine if complaints are well-written (considered desirable

In [42]:
#Equal Complaint Synthesis  
for i in range(len(random_samples[0:5])):
    instructions = [
        {
            "role": "You are an assistant who is generating short synthesized customer complaints for banks.", 
            
            "content": f"""
            Create synthetic complaints that retain the core message and sentiment of the original complaints but change some words or phrases while ensuring there is no change in the overall meaning in at most 50 words.
            Make sure the sentences are complete, and that the output complaint is different from the original complaint. 
            
            Complaints: '''{random_samples[i]}'''"""
        } 
    ]
    prompt = format_instructions(instructions)
    payload = {
        "inputs": prompt, 
        "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.5}
    }
    response = query_endpoint(payload)
    print_instructions(prompt, response)

> Input
<s>[INST] Create synthetic complaints that retain the core message and sentiment of the original complaints but change some words or phrases while ensuring there is no change in the overall meaning in at most 50 words.
            Make sure the sentences are complete, and that the output complaint is different from the original complaint. 
            
            Complaints: '''This is a follow-up of complaint XXXX. FIA Card Services has never sent me their definition of " cash equivalent '' or anything else. In addition, to harass me further, the last statement included a " minimum interest charge '' of {$1.00}. I owe them nothing and that charge was so absurd that their automated line took it off. FIA cheat and lies and is totally irresponsible. There should be some penalties or they should at least tell the truth. They record their calls so al this can be easily verified.''' [/INST] 

> Output
1. I am writing to follow up on my previous complaint about FIA Card Services. De

In [43]:
#Robustness
%%time 

for i in range(len(random_samples[0:5])):
    instructions = [
        {
            "role": "system",
            "content": "Your task is to determine if customer complaints from banks are well written without any issues \
            Provide modified sample",
        },
        {
            "role": "user",
            "content": f"""
            Determine if complaints are well-written (considered desirable), \
            exhibit severe grammar or punctuation issues (considered problematic), \
            or contain irrelevant, frivolous information (also considered problematic). \
            Make your response as short as possible
            
            Complaints: '''{random_samples[i]}'''"""
        }
    ]
    prompt = format_instructions(instructions)
    payload = {
        "inputs": prompt,
        "parameters": {"max_new_tokens": 260, "temperature": 0.5, "top_p": 0.9}
    }
    response = query_endpoint(payload)
    
    print(i, response, "\n")

0 [{'generated_text': '1. The complaint is well-written and relevant to the issue.\n2. There are minor grammar and punctuation issues, but they do not significantly affect the clarity or meaning of the complaint.\n3. The complaint does not contain irrelevant or frivolous information.'}] 

1 [{'generated_text': '1. Complaints are well-written and relevant. No severe grammar or punctuation issues were identified. No frivolous information was present.'}] 

2 [{'generated_text': '1. Well-written: This complaint is well-written and clearly states the issue at hand.\n2. No severe grammar or punctuation issues: The complaint is grammatically correct and does not contain any significant punctuation errors.\n3. No irrelevant or frivolous information: The complaint is relevant and contains important information about the situation.'}] 

3 [{'generated_text': '1. Complaints are well-written.'}] 

4 [{'generated_text': '1. The complaints are well-written, as they clearly state the issue and provid

In [45]:
#Invariance  
for i in range(len(random_samples[0:5])):
    instructions = [
        {
            "role": "You are an assistant who is generating short synthesized customer complaints for banks.", 
            
            "content": f"""
            Create synthetic complaints that retain the core message and sentiment of the original complaints but change some words or phrases while ensuring there is no change in the overall meaning.
            Keep your response, at most, 50 words. 
            Make sure the sentences are complete, and that the output complaint is different from the original complaint. 
            
            Complaints: '''{random_samples[i]}'''"""
        } 
    ]
    prompt = format_instructions(instructions)
    payload = {
        "inputs": prompt, 
        "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.5}
    }
    response = query_endpoint(payload)
    print_instructions(prompt, response)

> Input
<s>[INST] Create synthetic complaints that retain the core message and sentiment of the original complaints but change some words or phrases while ensuring there is no change in the overall meaning.
            Keep your response, at most, 50 words. 
            Make sure the sentences are complete, and that the output complaint is different from the original complaint. 
            
            Complaints: '''This is a follow-up of complaint XXXX. FIA Card Services has never sent me their definition of " cash equivalent '' or anything else. In addition, to harass me further, the last statement included a " minimum interest charge '' of {$1.00}. I owe them nothing and that charge was so absurd that their automated line took it off. FIA cheat and lies and is totally irresponsible. There should be some penalties or they should at least tell the truth. They record their calls so al this can be easily verified.''' [/INST] 

> Output
1. This is a follow-up to issue XXX. FIA Card Ser

In [46]:
#Harshness 
for i in range(len(random_samples[0:5])):
    instructions = [
        {
            "role": "You are an assistant who is generating short synthesized customer complaints for banks.", 
            
            "content": f"""
            Develop a method to make complaints more harsh or less harsh while keeping the underlying issue intact in at most 50 words.
            Use a range of intensity levels (Less Harsh or More harsh) to ensure diversity. 
            
            Complaints: '''{random_samples[i]}'''"""
        } 
    ]
    prompt = format_instructions(instructions)
    payload = {
        "inputs": prompt, 
        "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.8}
    }
    response = query_endpoint(payload)
    print_instructions(prompt, response)

> Input
<s>[INST] Develop a method to make complaints more harsh or less harsh while keeping the underlying issue intact in at most 50 words.
            Use a range of intensity levels (Less Harsh or More harsh) to ensure diversity. 
            
            Complaints: '''This is a follow-up of complaint XXXX. FIA Card Services has never sent me their definition of " cash equivalent '' or anything else. In addition, to harass me further, the last statement included a " minimum interest charge '' of {$1.00}. I owe them nothing and that charge was so absurd that their automated line took it off. FIA cheat and lies and is totally irresponsible. There should be some penalties or they should at least tell the truth. They record their calls so al this can be easily verified.''' [/INST] 

> Output
1. Less Harsh: FIA Card Services could improve their communication by sending a clearer definition of "cash equivalent." I appreciate their efforts to minimize further harassment by taking off the